In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
sample_submit_df = pd.read_csv('./data/sample_submit.csv')

In [4]:
train_df = train_df.drop(columns=['id'])

In [5]:
train_df.head()

,age,sex,bmi,children,smoker,region,charges
0,26,male,32.665465,3,no,southeast,0
1,41,male,29.798725,1,no,southwest,0
2,28,male,32.722029,0,yes,northwest,1
3,20,female,38.429831,2,no,southeast,0
4,45,female,29.641854,1,no,northwest,0


In [6]:
X = pd.get_dummies(train_df[['age', 'sex', 'bmi', 'children', 'smoker', 'region']])
y = train_df['charges']

In [7]:
X.head()

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,26,32.665465,3,0,1,1,0,0,0,1,0
1,41,29.798725,1,0,1,1,0,0,0,0,1
2,28,32.722029,0,0,1,0,1,0,1,0,0
3,20,38.429831,2,1,0,1,0,0,0,1,0
4,45,29.641854,1,1,0,1,0,0,1,0,0


In [8]:
X.shape, y.shape

((1600, 11), (1600,))

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [10]:
X_train.shape , y_train.shape

((1280, 11), (1280,))

# SVM(one versus one V.S. one versus the rest)

In [11]:
from sklearn.datasets import load_digits
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC


In [12]:
C = 10.
kernel = 'poly'
gamma  = 0.01
degree=3

In [13]:
scv = SVC(C=C, kernel=kernel, gamma=gamma, degree= degree)
scv.fit(X_train, y_train)
y_pred_svc = scv.predict(X_test)

In [14]:
estimator = SVC(C=C, kernel=kernel, gamma=gamma)
scv_therest = OneVsRestClassifier(estimator)
scv_therest.fit(X_train, y_train)
y_pred_therest = scv_therest.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print ('One-versus-the-rest: {:.5f}'.format(accuracy_score(y_test, y_pred_therest)))
print ('One-versus-one: {:.5f}'.format(accuracy_score(y_test, y_pred_svc)))

In [ ]:
from sklearn.metrics import f1_score
print('f1 score: {:.3f}'.format(f1_score(y_test, y_pred_therest, average='macro')))

# グリッドサーチ

In [ ]:
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [ ]:
param_grid = [0.01, 0.1, 1., 10., 100.]
svc_param = {
    "C":param_grid,
    "gamma":param_grid
}

In [ ]:
# スコア方法をF1に設定
def f1score(y_train: np.array, y_pred: np.array):
    score = f1_score(y_test, clf.predict(X_test),average='macro')
    return score


f1_scoring = make_scorer(f1score, greater_is_better=True)

In [ ]:
svc=SVC()

In [ ]:
# グリッドサーチで学習
svc_search = GridSearchCV(estimator=svc, param_grid=svc_param, scoring='accuracy', cv=5)
svc_search.fit(X_train, y_train)

In [ ]:
print('Best parameters: {}'.format(svc_search.best_params_))
print('Best score: {:.3f}'.format(svc_search.best_score_))

In [ ]:
clf = svc_search.best_estimator_
clf.fit(X_train, y_train)

In [ ]:
print('f1 score: {:.3f}'.format(f1_score(y_test, clf.predict(X_test),average='macro')))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_range = np.power(10, np.arange(start=-3, stop=3, step=0.1))
param_distributions = {"C":param_range, "gamma":param_range}
svc = SVC()
rs = RandomizedSearchCV(estimator=svc, param_distributions=param_distributions, scoring="accuracy", cv=5, n_iter=100)
rs.fit(X_train, y_train)

In [ ]:
rs.best_score_, rs.best_params_

In [ ]:
clf = svc_search.best_estimator_
clf.fit(X_train, y_train)

In [ ]:
print('f1 score: {:.3f}'.format(f1_score(y_test, clf.predict(X_test),average='macro')))